In [1]:
import cv2
import numpy as np
import ipywidgets
from IPython.display import display
import numpy as np
import base64
import time

In [2]:
print(cv2.__version__)

4.8.0-dev


In [3]:
def get_faces(img, device='cpu'):
    if device == 'cpu':
        face_cascades = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        return face_cascades.detectMultiScale(img, 1.3, 5)
    elif device == 'gpu':
        face_cascades = cv2.cuda_CascadeClassifier.create('./cascades_cuda/haarcascade_frontalface_alt.xml')
        faces = face_cascades.detectMultiScale(img).download()
        if faces is None:
            return []
        return faces[0]

In [4]:
def blur_faces(faces, color_img, gray_img):
    for (x, y, w, h) in faces:
        cv2.rectangle(color_img,pt1=(x,y),pt2=(x+w,y+h),color=(255,0,0),thickness=2)
        roi_gray = gray_img[y:y+h, x:x+w]
        roi_color = color_img[y:y+h, x:x+w]
        roi = cv2.GaussianBlur(roi_color, (23, 23), 30)

        color_img[y:y+w, x:x+h] = roi
    return color_img

In [5]:
def face_reduction(color_img, gray_img, device='cpu'):
    faces = get_faces(gray_img, device)
    
    gray_img = gray_img.download() if device=='gpu' else gray_img
    color_cpu_img = color_img.download() if device=='gpu' else color_img
    blur_face = blur_faces(faces, color_cpu_img, gray_img)
    return blur_face

### Face Redaction with CPU

In [6]:
%%time
c_widget = ipywidgets.Image(format='jpeg')
display(c_widget)

time_taken = []

cap = cv2.VideoCapture('hamilton_clip.mp4')

while True:
    try:
        ret, frame = cap.read()
        if not ret:
            print(f"Can't receive frames...")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        start_time = time.time()
        frame = face_reduction(frame, gray, device='cpu')
        end_time = time.time()
        time_taken.append(end_time - start_time)

        c_widget.value = bytes(cv2.imencode('.jpg', frame)[1])
    
    except Exception as e:
        print(e)
        break

cap.release()
print(f"Time taken for face recuction in CPU: {sum(time_taken)} sec")
# cv2.destroyAllWindows()

Image(value=b'', format='jpeg')

Can't receive frames...
Time taken for face recuction in CPU: 147.63707184791565 sec
CPU times: total: 4min 59s
Wall time: 2min 41s


### Face Reduction with GPU

In [7]:
%%time
rimage_widget = ipywidgets.Image(format='jpeg')

display(rimage_widget)

cap = cv2.VideoCapture('hamilton_clip.mp4')
time_taken = []

while True:
    # try:
        ret, frame = cap.read()

        if not ret:
            print("Can't retrive frame ...")
            break

        img_gpu = cv2.cuda_GpuMat()
        img_gpu.upload(frame)
        gray = cv2.cuda.cvtColor(img_gpu, cv2.COLOR_BGR2GRAY)

        start_time = time.time()
        frame = face_reduction(img_gpu, gray, device='gpu')
        end_time = time.time()
        time_taken.append(end_time - start_time)
        
        rimage_widget.value = bytes(cv2.imencode('.jpg', frame)[1])
    # except Exception as e:
    #     print(e)
    #     break

cap.release()

print(f"Time taken for face recuction in GPU: {sum(time_taken)} sec")
# cv2.destroyAllWindows()


Image(value=b'', format='jpeg')

Can't retrive frame ...
Time taken for face recuction in GPU: 280.1159393787384 sec
CPU times: total: 4min 51s
Wall time: 4min 56s


In [8]:
ipywidgets.Widget.close_all()